In [ ]:
!pip install --quiet simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 392.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from simpletransformers.t5 import T5Model

In [ ]:
temp = pd.read_csv("/content/drive/MyDrive/QASPER_openAI.csv")
temp_train= pd.DataFrame()

temp=temp.replace(['Answer 1:',],'', regex=True)
temp=temp.replace(['Question 1:',],'', regex=True)
temp=temp.replace(['Question:',],'', regex=True)
temp=temp.replace(['Answer:',],'', regex=True)
temp=temp.replace(['Question :',],'', regex=True)
temp=temp.replace(['Answer :',],'', regex=True)

temp_train["input_text"] = temp['answer']
temp_train["target_text"] = temp['question']
# temp_train["context"] = temp['context']
temp_train["prefix"] = "ask_question:"

In [ ]:
temp_train.shape

(20153, 3)

In [ ]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 8,
    "num_train_epochs": 1,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "fp16": False,
}
model = T5Model("t5", "t5-base", args=model_args, overwrite_output_dir = True)

In [ ]:
model.args

T5Args(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=False, adafactor_scale_parameter=False, adafactor_warmup_init=False, adam_betas=(0.9, 0.999), adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=False, gradient_accumulation_steps=1, learning_rate=0.001, local_rank=-1, logging_steps=50, loss_type=None, loss_args={}, manual_seed=None, ma

In [ ]:
model.train_model(temp_train)

  0%|          | 0/20153 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2520 [00:00<?, ?it/s]

(2520, 0.2889789280257121)

In [ ]:
preds = model.predict(
    
    ["ask_question: " + "The decoder is also composed of a stack of N=6 identical layers. \
    In addition to the two sub-layers in each encoder layer, the decoder inserts a third \
    sub-layer, which performs multi-head attention over the output of the encoder stack.\
    Similar to the encoder, we employ residual connections around each of the sub-layers,\
    followed by layer normalization. We also modify the self-attention sub-layer in the \
    decoder stack to prevent positions from attending to subsequent positions. \
    This masking, combined with the fact that the output embeddings are offset by one \
    position, ensures that the predictions for position i can depend only on the known \
    outputs at positions less than i"]
)

print(preds)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['How does the decoder layer differ from the encoder stack?']


In [ ]:
model.save_model('/content/T5_question_model')

In [ ]:
import pickle

pickle.dump(model, open("T5_question_model.pkl", "wb"))

In [ ]:
no_fine_tune_model = T5Model("t5", "t5-base", args=model_args, overwrite_output_dir = True)

In [ ]:
preds = no_fine_tune_model.predict(
    
    ["ask_question: " + "Figure 1 illustrates an overview of our model, which is mainly consisted of three parts: word-level module, sentence-level module, and pair-level module. Token sequences of sentence pairs (Arg1 and Arg2) are encoded by word-level module first and every token becomes a word embedding augmented by subword and ELMo. Then these embeddings are fed to sentence-level module and processed by stacked encoder blocks (CNN or RNN encoder block). Every block layer outputs representation for each token. Furthermore, the output of each layer is processed by bi-attention module in the pair-level module, and concatenated to pair representation, which is finally sent to classifiers which are multiple layer perceptrons (MLP) with softmax. The model details are given in the rest of this section."]
)

print(preds)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['True']


In [ ]:
context = "The decoder is also composed of a stack of N=6 identical layers. In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections around each of the sub-layers, followed by layer normalization. We also modify the self-attention sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This masking, combined with the fact that the output embeddings are offset by one position, ensures that the predictions for position i can depend only on the known outputs at positions less than i."
question = "What modifications were made to the self-attention sub-layer in the decoder stack of the Transformer model described in the given text, and what is the purpose of these modifications?"

input_str = f'context: {context}  question: {question}'

outputs = model.predict([input_str])
answer = outputs[0]

print(answer)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

How does the decoder stack differ from the encoder stack?
